In [ ]:
import pandas as pd
import hvplot.pandas
import panel as pn

df = pd.read_csv('world_wide_COVID_data_daily_data.csv')
df = df[df['year'] == 2020][::-1]

#获取所有洲名
continentExp = list(set(df.continentExp.values))
continentExp.sort()

#定义widgets:continents, months, countries
continents = pn.widgets.Select(name = 'Continents',options = continentExp)
months = pn.widgets.IntRangeSlider(name='Months',start=1, end=12, step=1,value=(3,4))
countries = pn.widgets.Select(name = 'Countries')
button = pn.widgets.RadioButtonGroup(name='Radio Button Group', options=['cases', 'deaths'])   #两个按钮:cases和deaths

#function: update countries
@pn.depends(continents, watch = True)
def update_countries(continents):
    table = df[df['continentExp'] == continents]       #筛选df中符合continents的数据
    country_name = list(set(table.countriesAndTerritories.values))      #获取continents对应的country name
    country_name.sort()         #按字母排序
    countries.options = country_name       #更新countries的options为新的country name
    return countries

#function: table
@pn.depends(countries,months)
def make_table(countries,months):
    months = range(months[0],(months[1]+1))
    table = df[df['countriesAndTerritories'] == countries]     #选中country，df中对应的数据
    table = table[table['month'].isin(months)]      #选取month，table中对应的数据
    return table.hvplot.table(width = 800)          #hvplot绘制表格

#function: make plot
@pn.depends(countries,months,button)
def make_plot(countries,months,button):
    months = range(months[0],(months[1]+1))
    table = df[df['countriesAndTerritories'] == countries]     #选中country，df中对应的数据
    table = table[table['month'].isin(months)]      #选取month，table中对应的数据
    plot = table.hvplot.line(x = 'dateRep',rot=90, y = button)   
    return plot
    
pn.Row(pn.Column(continents,countries,months,button,width = 200),pn.Column(make_plot,make_table))